In [ ]:
#@title Start-up Step 1: XLS and OpenRoad scripts {run:"auto"}

!rm -rf *

# Import required Python libraries
import os
import pathlib
import sys
import jinja2
import IPython.display
import PIL.Image
import graphviz
import pathlib

from IPython.display import display, display_png

# Set Stable XLS Version for classroom environment

xls_version = 'v0.0.0-5984-g5eed6363a' #@param {type:"string"}
xls_colab_version = 'v0.0.0-5639-g02334a86c' #@param {type:"string"}

!echo '📦 downloading xls-{xls_version}'
!curl --show-error -L https://github.com/google/xls/releases/download/{xls_version}/xls-{xls_version}-linux-x64.tar.gz | tar xzf - --strip-components=1
!echo '🧪 setting up colab integration xls_colab-{xls_colab_version}'
!python -m pip install --quiet --no-cache-dir --ignore-installed https://github.com/proppy/xls/releases/download/{xls_colab_version}/xls_colab-0.0.0-py3-none-any.whl
import xls.contrib.colab
_ = xls.contrib.colab.register_dslx_magic()

!python -m pip install logger
!python -m pip install colabtools
import logger

# Must verify xls_work_dir is created
!if test -d xls_work_dir; then echo "xls_work_dir exists"; else mkdir xls_work_dir;  fi

#@title  First Run Only #4 - OpenRoad Setup {run:"auto"}
!echo '🛣️ setting up openroad'
yosys_version = '0.38_93_g84116c9a3' #@param {type:"string"}
openroad_version = '2.0_12381_g01bba3695' #@param {type:"string"}
rules_hdl_version = '2eb050e80a5c42ac3ffdb7e70392d86a6896dfc7' #@param {type:"string"}

# Install stable OpenROAD Version
!echo '🛣️ installing openroad and friends'
!curl -L -O https://repo.anaconda.com/miniconda/Miniconda3-py310_24.1.2-0-Linux-x86_64.sh
!bash Miniconda3-py310_24.1.2-0-Linux-x86_64.sh -b -p conda-env/
import pathlib
conda_prefix_path = pathlib.Path('conda-env')
CONDA_PREFIX = str(conda_prefix_path.resolve())
%env CONDA_PREFIX={CONDA_PREFIX}
!conda-env/bin/conda install -yq -c "litex-hub" openroad={openroad_version} yosys={yosys_version}

!python -m pip install gdstk==0.9.41 tqdm

!gsutil cp gs://proppy-eda/pdk_info_asap7.zip .
!gsutil cp gs://proppy-eda/pdk_info_sky130.zip .

!unzip -q -o pdk_info_asap7.zip
!unzip -q -o pdk_info_sky130.zip

!echo '🧰 generating PDK metadata'
!curl --show-error -L  https://github.com/hdl/bazel_rules_hdl/archive/{rules_hdl_version}.tar.gz | tar xzf - --strip-components=1
!curl -L -O https://github.com/protocolbuffers/protobuf/releases/download/v24.3/protoc-24.3-linux-x86_64.zip
!unzip -q -o protoc-24.3-linux-x86_64.zip
!{sys.executable} -m pip install protobuf

!echo '📁 organizing PDK for XLS and OpenROAD Flows'
!wget https://raw.githubusercontent.com/mmorri22/cse30321/main/xls/xls_setup.py
!wget https://raw.githubusercontent.com/mmorri22/cse30321/main/xls/sky130_data_pdk_info.textproto
!python xls_setup.py
!mv /content/sky130_data_pdk_info.textproto /content/com_google_skywater_pdk_sky130_fd_sc_hd/sky130_data_pdk_info.textproto
!echo '🖼️ Setup for viewing 3D GDSII File'
!python -m pip install numpy
!python -m pip install gdspy
!python -m pip install numpy-stl
!python -m pip install triangle
!python -m pip install k3d

# gdspy is used to open the gds file
import gdspy

# Used to write the output stl file (Why we installed numpy-stl)
from stl import mesh

# Using numpy will permit fast calculations on lots of points
import numpy as np
import matplotlib

# Required to triangulate polygons
import triangle

# To render in 3d
import k3d

📦 downloading xls-v0.0.0-5984-g5eed6363a
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 60.2M  100 60.2M    0     0  7605k      0  0:00:08  0:00:08 --:--:-- 8103k
🧪 setting up colab integration xls_colab-v0.0.0-5639-g02334a86c
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.6/189.6 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for logger: filename=logger-1.4-py3-none-any.whl size=1759 sha256=c9e46412d713fb142487c111a0d04a31526a1b6ab07a02fea283ed5f97ec40f1
  Stored in directory: /root/.cache/pip/wheels/cd/1d/c8/f1361043ff09fd3c9a747b199e8e37bd716bacf0843bbdb68f
Successfully built logger
xls_work_dir exists
🛣️ setting up openroad
🛣️ installing openroad and friends
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                   

In [ ]:
#@title Start-up Step 2: Select your PDK {run:"auto"}

pdk = 'sky130' #@param ["asap7", "sky130"] {allow-input: false}

xls.contrib.colab.pdk = pdk


#@title Select your PDK {run:"auto"}

!bin/protoc --python_out=. pdk/proto/pdk_info.proto
!ln -sf pdk/proto/pdk_info_pb2.py
import pdk_info_pb2

import enum
import dataclasses
import json
import pathlib
import subprocess
from typing import Any, Callable, Dict, Optional, Union

from google.colab import widgets
from google.protobuf import text_format
import pandas as pd

yosys = conda_prefix_path / 'bin/yosys'
openroad = conda_prefix_path / 'bin/openroad'
yosys_tcl = 'synthesis/synth.tcl'

default_work_dir = xls.contrib.colab.default_work_dir

def pdk_info_proto(
    path: pathlib.Path, optional: bool = False
) -> Optional[pdk_info_pb2.PdkInfoProto]:
  """Load PDK info from prototext.

  Args:
    path: path to prototext file.
    optional: if True, failure to access the pdk info will not produce an error.

  Returns:
    Decoded pdk info proto or None if optional.
  """
  if optional and not path.exists():
    return None
  with path.open('r') as f:
    proto = pdk_info_pb2.PdkInfoProto()
    text_format.Parse(f.read(), proto)
    return proto

pdks = {

    'asap7': {
        'delay_model': 'asap7',
        'pdk_info': pdk_info_proto(
            pathlib.Path('asap7/asap7_data_pdk_info.textproto'),
        ),
    },

    'sky130': {
        'delay_model': 'sky130',
        'pdk_info': pdk_info_proto(
            pathlib.Path('com_google_skywater_pdk_sky130_fd_sc_hd/sky130_data_pdk_info.textproto'),
        ),
    },
}

@dataclasses.dataclass(frozen=True)
class RelativeCoreArea:
  utilization_percent: float


@dataclasses.dataclass(frozen=True)
class AbsoluteCoreArea:
  core_width_microns: int
  core_padding_microns: int


@enum.unique
class ImplementationStep(enum.Enum):
  """Steps in the implementation flow."""

  XLS = 'xls'
  SYNTHESIS = 'synthesis'
  PLACEMENT = 'placement'


class PdkRuntimeError(RuntimeError):
  pass


class OpenroadRuntimeError(RuntimeError):
  pass


class OpenstaRuntimeError(RuntimeError):
  pass


class YosysRuntimeError(RuntimeError):
  pass


@dataclasses.dataclass(frozen=True)
class SynthesisResults:
  synth_v: pathlib.Path
  design_stats: pd.DataFrame
  cell_stats: pd.DataFrame


def run_synthesis(
    *,
    selected_pdk: Optional[str] = None,
    work_dir: pathlib.Path = default_work_dir,
    silent: bool = False,
) -> SynthesisResults:
  """Run synthesis with Yosys.

  Args:
    selected_pdk: The pdk to use.
    work_dir: Directory that contains verilog and will be where outputs are put.
    silent: Suppress output.

  Returns:
    Metrics from running synthesis.

  Raises:
    PdkRuntimeError: on PDK error.
    YosysRuntimeError: on yosys error.
  """
  if selected_pdk is None:
    selected_pdk = pdk
  pdk_info = pdks[selected_pdk]['pdk_info']
  if pdk_info is None:
    raise PdkRuntimeError(f'PDK "{selected_pdk}" is restricted')

  liberty = (pathlib.Path(pdk) / pathlib.Path(pdk_info.liberty_path).name).resolve()
  synth_v = (work_dir / 'user_module_synth.v').resolve()
  synth_v_flist = (work_dir / 'user_module_synth_v.flist').resolve()
  synth_uhdm_flist = (work_dir / 'user_module_synth_uhdm.flist').resolve()
  synth_uhdm_flist.touch()
  synth_stats_json = (work_dir / 'user_module_synth_stats.json').resolve()
  dont_use_args = ' '.join(
      f'-dont_use {pat}'
      for pat in pdk_info.do_not_use_cell_list
  )
  # run yosys synthesis
  with synth_v_flist.open('w') as f:
    top_v = work_dir / 'user_module.sv'
    f.write(str(top_v.resolve()))
  !FLIST='{synth_v_flist}' ABC_SCRIPT='' CONSTR='' TOP='user_module' OUTPUT='{synth_v}' UHDM_FLIST='{synth_uhdm_flist}' LIBERTY='{liberty}' STATS_JSON='{synth_stats_json}' DONT_USE_ARGS='{dont_use_args}' {yosys} -c '{yosys_tcl}'
  with synth_stats_json.open('r') as f:
    synth_stats = json.load(f)
  design_stats = synth_stats['design']
  cells_stats = design_stats.pop('num_cells_by_type')
  design_stats = pd.DataFrame.from_dict(
      design_stats, orient='index', columns=['cells']
  )
  cells_stats = pd.DataFrame.from_dict(
      cells_stats, orient='index', columns=['stats']
  )

  return SynthesisResults(
      synth_v=synth_v, design_stats=design_stats, cell_stats=cells_stats
  )


def run_opensta(
    *,
    selected_pdk: Optional[str] = None,
    work_dir: pathlib.Path = default_work_dir,
    silent: bool = False,
) -> pd.DataFrame:
  """Run OpenSta and collect timing metrics.

  Args:
    selected_pdk: The pdk to use.
    work_dir: Directory that contains verilog.
    silent: Suppress output.

  Returns:
    Dataframe containing timing report.

  Raises:
    OpenstaRuntimeError: on OpenSTA error.
    PdkRuntimeError: on PDK error.
  """
  if selected_pdk is None:
    selected_pdk = pdk
  pdk_info = pdks[selected_pdk]['pdk_info']
  if pdk_info is None:
    raise PdkRuntimeError(f'PDK "{selected_pdk}" is restricted')

  liberty = pathlib.Path(pdk) / pdk_info.liberty_path
  tech_lef = pathlib.Path(pdk) / pdk_info.tech_lef_path
  read_cell_lefs = '\n'.join(
      f'read_lef {pathlib.Path(pdk) / cell_lef_path}'
      for cell_lef_path in pdk_info.cell_lef_paths
  )
  synth_v = work_dir / 'user_module_synth.v'
  top = 'user_module'
  opensta_log = work_dir / 'user_module_sta.log'

  openroad_script = f"""
  sta::redirect_file_begin {opensta_log}
  read_lef {tech_lef}
  {read_cell_lefs}
  read_liberty {liberty}
  read_verilog {synth_v}
  link_design  {top}
  report_checks -unconstrained
  sta::redirect_file_end
  """
  openroad_tcl = work_dir / 'openroad_sta.tcl'
  with openroad_tcl.open('w') as f:
    f.write(openroad_script)

  # run opensta static timing analysis
  !{openroad} {openroad_tcl} -exit

  columns = ['delay', 'time', 'edge', 'net', 'gate']

  import re
  def sta_report_paths(opensta_log):
    with open(opensta_log) as f:
      sta_report = f.read()
    m = re.search(r'---+(.*)---+', sta_report, flags=re.M | re.S)
    for path in m.group(1).split('\n')[1:-2]:
      parts = path.split(None, maxsplit=len(columns) - 1)
      yield float(parts[0]), float(parts[1]), parts[2], parts[3], parts[4]

  df = pd.DataFrame.from_records(sta_report_paths(opensta_log), columns=columns)
  df['gate'] = df['gate'].str.replace('[()]', '', regex=True)

  return df


@dataclasses.dataclass(frozen=True)
class PlacementResults:
  openroad_global_placement_layout: pathlib.Path
  area: pd.DataFrame
  metrics: pd.DataFrame
  power: pd.DataFrame


def run_placement(
    *,
    clock_period_ps: int,
    placement_density: float,
    core_area: Union[RelativeCoreArea, AbsoluteCoreArea],
    selected_pdk: Optional[str] = None,
    work_dir: pathlib.Path = default_work_dir,
    silent: bool = False,
) -> PlacementResults:
  """Run OpenRoad placement.

  Args:
    clock_period_ps: Clock period in picoseconds.
    placement_density: Placement density in [0.0, 1.0].
    core_area: Relative or absolute core area specification.
    selected_pdk: The pdk to use.
    work_dir: Directory that contains verilog and will be where outputs are put.
    silent: Suppress output.

  Returns:
    Outputs from running placement.

  Raises:
    OpenroadRuntimeError: on OpenRoad error.
    OpenstaRuntimeError: on OpenSTA error.
    PdkRuntimeError: on PDK error.
    ValueError: on invalid inputs.
    YosysRuntimeError: on yosys error.
  """
  clock_period_ns = clock_period_ps / 1000.0
  if selected_pdk is None:
    selected_pdk = pdk
  pdk_info = pdks[selected_pdk]['pdk_info']
  if pdk_info is None:
    raise PdkRuntimeError(f'PDK "{selected_pdk}" is restricted')

  liberty = pathlib.Path(pdk) / pdk_info.liberty_path
  tech_lef = pathlib.Path(pdk) / pdk_info.tech_lef_path
  read_cell_lefs = '\n'.join(
      f'read_lef {pathlib.Path(pdk) / cell_lef_path}'
      for cell_lef_path in pdk_info.cell_lef_paths
  )

  if isinstance(core_area, AbsoluteCoreArea):
    die_side_microns = (
        core_area.core_width_microns + core_area.core_padding_microns * 2
    )
    core_side_microns = (
        core_area.core_width_microns + core_area.core_padding_microns
    )
    initialize_floorplan_args = (
        f' -die_area "0 0 {die_side_microns} {die_side_microns}" -core_area'
        f' "{core_area.core_padding_microns} {core_area.core_padding_microns} {core_side_microns} {core_side_microns}"'
    )
  elif isinstance(core_area, RelativeCoreArea):
    initialize_floorplan_args = (
        f' -utilization {core_area.utilization_percent} -aspect_ratio 1.0'
    )
  else:
    raise ValueError(
        'Expected core_area to be AbsoluteCoreArea or RelativeCoreArea, got'
        f' {core_area!r}'
    )

  initialize_floorplan_command = (
      f'initialize_floorplan -site "{pdk_info.cell_site}"'
      f' {initialize_floorplan_args}'
  )

  def source_pdk_info_tcl(path):
    return f'source {pathlib.Path(pdk) / path}' if path else ''

  source_tracks_file = source_pdk_info_tcl(pdk_info.tracks_file_path)
  source_rc_script_configuration = source_pdk_info_tcl(
      pdk_info.rc_script_configuration_path
  )
  source_pdn_config = source_pdk_info_tcl(pdk_info.pdn_config_path)
  if pdk_info.tapcell_tcl_path:
    tapcell_command = source_pdk_info_tcl(pdk_info.tapcell_tcl_path)
  else:
    tapcell_command = (
        f'tapcell -distance {pdk_info.tapcell_distance} -tapcell_master'
        f' {pdk_info.tap_cell}'
    )

  synth_v = work_dir / 'user_module_synth.v'
  openroad_metrics = work_dir / 'openroad_metrics.json'
  openroad_global_placement_layout = work_dir / 'openroad_global_placement.png'

  openroad_script = f"""
  read_lef {tech_lef}
  {read_cell_lefs}
  read_liberty {liberty}
  read_verilog {synth_v}
  link_design user_module
  {initialize_floorplan_command}
  {source_tracks_file}
  insert_tiecells {pdk_info.tie_high_port} -prefix "TIE_ONE_"
  insert_tiecells {pdk_info.tie_low_port} -prefix "TIE_ZERO_"
  create_clock [get_ports clk] -period {clock_period_ns}
  {source_rc_script_configuration}
  set_wire_rc -signal -layer "{pdk_info.wire_rc_signal_metal_layer}"
  set_wire_rc -clock  -layer "{pdk_info.wire_rc_clock_metal_layer}"
  place_pins -hor_layers {pdk_info.pin_horizontal_metal_layer} -ver_layers {pdk_info.pin_vertical_metal_layer}
  {tapcell_command}
  {source_pdn_config}
  pdngen -verbose
  global_placement -timing_driven -routability_driven -density {placement_density} -pad_left {pdk_info.global_placement_cell_pad} -pad_right {pdk_info.global_placement_cell_pad}
  remove_buffers
  estimate_parasitics -placement
  repair_design
  repair_timing
  utl::metric "utilization_percent" [rsz::utilization]
  utl::metric "design_area" [rsz::design_area]
  utl::metric "power" [sta::design_power [sta::parse_corner {{}}]]
  utl::metric "wns" [sta::worst_slack -max]
  report_power
  report_design_area
  if {{[info procs save_image] == "save_image"}} {{
    save_image -resolution 0.005 "{openroad_global_placement_layout}"
  }}
  """
  openroad_tcl = work_dir / 'place.tcl'
  with openroad_tcl.open('w') as f:
    f.write(openroad_script)
  !QT_QPA_PLATFORM=minimal {openroad} -metrics {openroad_metrics} -exit {openroad_tcl}

  with open(work_dir / 'openroad_metrics.json', 'r') as f:
    metrics = json.loads(f.read())
  df_area = pd.DataFrame.from_dict(
      {
          'global placement': [
              float(metrics['design_area']) * 1e12,
              float(metrics['utilization_percent']) * 100,
          ]
      },
      columns=['area', 'utilization'],
      orient='index',
  )
  metrics_power = [float(m) * 1e6 for m in metrics['power'].split(' ')]
  df_power = pd.DataFrame().from_dict(
      {
          'sequential': metrics_power[4:8],
          'combinational': metrics_power[8:12],
          'clock': metrics_power[12:16],
          'macro': metrics_power[16:20],
          'pad': metrics_power[20:],
          'total': metrics_power[0:4],
      },
      orient='index',
      columns=['internal', 'switching', 'leakage', 'total'],
  )
  df_metrics = (
      pd.DataFrame.from_records([metrics])
      .transpose()
      .set_axis(['metrics'], axis=1)
  )
  return PlacementResults(
      openroad_global_placement_layout=openroad_global_placement_layout,
      area=df_area,
      metrics=df_metrics,
      power=df_power,
  )

In [ ]:
%%dslx --top=fsm_proc --pipeline_stages=1 --flop_inputs=false --flop_outputs=false --clock_period_ps=4650 --reset=reset
import std;

const E_TYPE = u16:0x0800;
const NEW_ORDER = u8:0x01;
const VERSION = u8:1;
const CID = u32:0x00000001;
const SID = u64:0x0000000000000002;
const FLAGS = u8:0;

struct STATE_RES{
    state:u4,
    read: u8,
    first:u64,
    regs: u64[7],
    count: u4,
    frame: bool,
    output: u64,
    delay: u4,
    order: u64
}

fn fsm_logic(
  state: u4,
  read: u8,
  first: u64,
  regs: u64[7],
  count: u4,
  frame: bool,
  output: u64,
  delay: u4,
  order: u64) -> STATE_RES {
  if (state == u4:0) {
        if(read == u8:0xe1){
            STATE_RES{state: u4:1, read, first, regs, count: u4:0, frame: false, output: u64:0x1, delay: u4:0, order}
        } else {
            STATE_RES{state: u4:0, read,first, regs, count: u4:0, frame: false, output:output, delay: u4:0, order}
        }
  }


  else if (state == u4:1) {
      if(delay == u4:7){
        if (count < u4:1) {
            let new_count = u4:1;
            let new_regs = update(regs, u4:0, first);
            STATE_RES{state, read,first, regs: new_regs, count: new_count, frame, output: u64:0x2, delay: u4:0, order }
        }
        else{
            let new_regs = update(regs, u4:1, first);
            STATE_RES{state: u4:2, read, first, regs: new_regs, count: u4: 0, frame, output: u64:0x2, delay: u4:0, order }
        }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, read, first, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:2) {
      if (delay == u4:7){
        let new_regs = update(regs, u4:2, first);
        STATE_RES{state: u4:3, read, first, regs: new_regs, count: u4: 0, frame, output: u64:3, delay: u4:0, order }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, read, first, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:3) {
      if (delay == u4:7){
        if (count < u4:1) {
            let new_count = count + u4:1;
            let new_regs = update(regs, u4:3, first);
            STATE_RES{state: u4:3, read, first, regs: new_regs, count: new_count, frame, output: u64:4, delay: u4:0, order }
        }
        else{
            let new_regs = update(regs, u4:3, first);
            STATE_RES{state: u4:4, read, first, regs: new_regs, count: u4:0, frame, output:u64:4, delay: u4:0, order }
        }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, read, first, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:4) {
      if (delay == u4:7){
        let new_regs = update(regs, u4:5, first);
        STATE_RES{state: u4:5, read, first, regs: new_regs, count: u4:0, frame, output: u64:5, delay: u4:0, order }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, read, first, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:5) {
      if (delay == u4:7){
        if (count < u4:1) {
            let new_count = count + u4:1;
            let new_regs = update(regs, u4:6, first);
            STATE_RES{state: u4:5, read, first, regs: new_regs, count: new_count, frame, output: u64:6, delay: u4:0, order }
        }
        else{
            let output = u64:0x555555555555555d;
            if (regs[5][8:16] != u8:4){
                STATE_RES{state: u4:15, read, first, regs, count: u4:0, frame, output: u64:0x888888888888888, delay: u4:0, order }
            }
            else{
                STATE_RES{state: u4:6, read, first, regs, count: u4:0, frame, output:output, delay: u4:0, order }
            }
       }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, read, first, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:6) {
      if (delay == u4:7){
        if (count == u4:0) {
            let new_count = count + u4:1;
            //let output = regs[0][48:64] ++ regs[1][0:32] ++ regs[0][0:16];
            STATE_RES{state: u4:6, read, first, regs, count: new_count, frame, output: u64:6, delay: u4:0, order }
        }
        else if (count == u4:1) {
            let new_count = count + u4:1;
            //let output = regs[0][16:48] ++ E_TYPE ++ regs[1][32:48];
            STATE_RES{state: u4:6, read, first, regs, count: new_count, frame, output: u64:6, delay: u4:0, order }
        }
        else  {
            //let output = regs[1][56:64] ++ regs[2][0:56];
            STATE_RES{state: u4:7, read, first, regs, count: u4:0, frame: bool: true, output: u64:6, delay: u4:0, order }
        }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, read, first, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:7) {
      if (delay == u4:7){
        if (frame){
            //let output = regs[2][48:64] ++ regs[3][0:8] ++ regs[4][24:40] ++ NEW_ORDER ++ VERSION ++ regs[4][40:48];
            STATE_RES{state: u4:8, read, first, regs, count: u4:0, frame, output: u64:7, delay: u4:0, order }
        } else {
            STATE_RES{state: u4:15, read, first, regs, count: u4:0, frame, output: u64:0x17 , delay: u4:0, order }
        }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, read, first, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:8) {
      if (delay == u4:7){
        if (count == u4:0) {
            let new_count = count + u4:1;
            //let output = regs[4][48:64] ++ regs[5][0:8] ++ CID ++ SID[56:64];
            STATE_RES{state: u4:8, read, first,  regs, count: new_count, frame, output: u64:8, delay: u4:0, order }
        }
        else if (count == u4:1) {
            let new_count = count + u4:1;
            //let output = SID[0:56] ++ order[0:8];
            STATE_RES{state: u4:8, read, first, regs, count: new_count, frame, output: u64:8, delay: u4:0, order }
        }
        else if(count == u4:2) {
            //let output = order[8:64] ++ regs[5][16:24];
            let new_count = count + u4:1;
            STATE_RES{state: u4:8, read, first, regs, count: new_count, frame, output: u64:8, delay: u4:0, order }
        } else if (count == u4:3) {
            //let output = regs[5][24:64] ++ regs[6][0:24];
            let new_count = count + u4:1;
            STATE_RES{state: u4:8, read, first, regs, count: new_count, frame, output: u64:8, delay: u4:0, order}
        } else {
           // let output = regs[6][24:56] ++ FLAGS ++ u24:0x000000;
            let new_order = order + u64:1;
            STATE_RES{state: u4:15, read, first, regs, count: u4:0, frame, output: u64:8, delay: u4:0, order: new_order}
        }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, read, first, regs, count, frame, output, delay: new_delay, order }
      }
  }
  else {
      if (delay == u4:7){
        STATE_RES{state: u4:0, read, first: u64:0, regs: [u64:0, u64:0, u64:0, u64:0, u64:0, u64:0, u64:0], count: u4:0, frame, output: u64:9, delay: u4:0, order}
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, read, first, regs, count: u4:0, frame, output, delay: new_delay, order }
      }
  }
}

proc fsm_proc{
    reading: chan<u8> in;
    output: chan<u8> out;

    init {
        STATE_RES{
          state: u4:0,
          read: u8:0,
          first: u64:0,
          regs: [u64:0, u64:0, u64:0, u64:0, u64:0, u64:0, u64:0],
          count: u4:0,
          frame: false,
          output: u64:0,
          delay: u4:0,
          order: u64:0
        }
    }

    config(reading: chan<u8> in, output: chan<u8> out) {
        (reading, output)
    }

    next(status:STATE_RES) {
        let tok = token();
        let (tok_reading, reading_in) = recv(tok, reading);
        let tok_in = join(tok_reading);

        let new_first = status.first[0:56] ++ reading_in;
        let output_out = status.output[0:8];
        let new_output = status.output[8:64] ++ u8:0;

        let status = fsm_logic(
          status.state,
          reading_in,
          new_first,
          status.regs,
          status.count,
          status.frame,
          new_output,
          status.delay,
          status.order
        );

        send(tok, output, output_out);

        status
    }
}


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
%%dslx --top=fsm_proc --pipeline_stages=1 --flop_inputs=false --flop_outputs=false --clock_period_ps=4650 --reset=reset
import std;
#test bench
const E_TYPE = u16:0x0800;
const NEW_ORDER = u8:0x01;
const VERSION = u8:1;
const CID = u32:0x00000001;
const SID = u64:0x0000000000000002;
const FLAGS = u8:0;

struct STATE_RES{
    state:u4,
    first:u64,
    second:u64,
    third:u64,
    regs: u64[7],
    count: u4,
    frame: bool,
    output: u64,
    delay: u4,
    order: u64
}

fn fsm_logic(
  state: u4,
  first: u64,
  second: u64,
  third: u64,
  regs: u64[7],
  count: u4,
  frame: bool,
  output: u64,
  delay: u4,
  order: u64) -> STATE_RES {
  if (state == u4:0) {
      if (delay == u4:7) {
        if(first == u64:0x555555555555555d){
            STATE_RES{state: u4:1, first, second, third, regs, count: u4:0, frame: false, output: u64:0x0, delay: u4:0, order}
        } else {
            STATE_RES{state: u4:0, first, second, third, regs, count: u4:0, frame: false, output: u64:0x0, delay: u4:0, order}
        }
      }
      else {
          let new_delay = delay + u4:1;
          STATE_RES{state: u4:0, first, second, third, regs, count: u4:0, frame: false, output, delay: new_delay, order}
      }
  }


  else if (state == u4:1) {
      if(delay == u4:7){
        if (count < u4:1) {
            let new_count = u4:1;
            STATE_RES{state, first, second, third, regs, count: new_count, frame, output: u64:0x0, delay: u4:0, order }
        }
        else{
            STATE_RES{state: u4:2, first, second, third, regs, count: u4: 0, frame, output: u64:0x0, delay: u4:0, order }
        }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:2) {
      if (delay == u4:7){
        let new_regs = update(regs, u4:0, third);
        let new_regs = update(new_regs, u4:1, second);
        let new_regs = update(new_regs, u4:2, first);
        STATE_RES{state: u4:3, first, second, third, regs: new_regs, count: u4: 0, frame, output: u64:0, delay: u4:0, order }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:3) {
      if (delay == u4:7){
        if (count < u4:1) {
            let new_count = count + u4:1;
            STATE_RES{state: u4:3, first, second, third, regs, count: new_count, frame, output: u64:0, delay: u4:0, order }
        }
        else{
            let new_regs = update(regs, u4:3, second);
            STATE_RES{state: u4:4, first, second, third, regs: new_regs, count: u4:0, frame, output:u64:0, delay: u4:0, order }
        }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:4) {
      if (delay == u4:7){
        let new_regs = update(regs, u4:4, second);
        let new_regs = update(new_regs, u4:5, first);
        STATE_RES{state: u4:5, first, second, third, regs: new_regs, count: u4:0, frame, output: u64:0, delay: u4:0, order }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:5) {
      if (delay == u4:7){
        if (count < u4:1) {
            let new_count = count + u4:1;
            let new_regs = update(regs, u4:6, first);
            STATE_RES{state: u4:5, first, second, third, regs: new_regs, count: new_count, frame, output: u64:0, delay: u4:0, order }
        }
        else{
            let output = u64:0x555555555555555d;
            if (regs[5][8:16] != u8:4){
                STATE_RES{state: u4:15, first, second, third, regs, count: u4:0, frame, output: u64:0x888888888888888, delay: u4:0, order }
            }
            else{
                STATE_RES{state: u4:6, first, second, third, regs, count: u4:0, frame, output:output, delay: u4:0, order }
            }
       }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:6) {
      if (delay == u4:7){
        if (count == u4:0) {
            let new_count = count + u4:1;
            let output = regs[0][48:64] ++ regs[1][0:32] ++ regs[0][0:16];
            STATE_RES{state: u4:6, first, second, third, regs, count: new_count, frame, output: output, delay: u4:0, order }
        }
        else if (count == u4:1) {
            let new_count = count + u4:1;
            let output = regs[0][16:48] ++ E_TYPE ++ regs[1][32:48];
            STATE_RES{state: u4:6, first, second, third, regs, count: new_count, frame, output: output, delay: u4:0, order }
        }
        else  {
            let output = regs[1][56:64] ++ regs[2][0:56];
            STATE_RES{state: u4:7, first, second, third, regs, count: u4:0, frame: bool: true, output: output, delay: u4:0, order }
        }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:7) {
      if (delay == u4:7){
        if (frame){
            let output = regs[2][48:64] ++ regs[3][0:8] ++ regs[4][24:40] ++ NEW_ORDER ++ VERSION ++ regs[4][40:48];
            STATE_RES{state: u4:8, first, second, third, regs, count: u4:0, frame, output: output, delay: u4:0, order }
        } else {
            STATE_RES{state: u4:15, first, second, third, regs, count: u4:0, frame, output: u64:0 , delay: u4:0, order }
        }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:8) {
      if (delay == u4:7){
        if (count == u4:0) {
            let new_count = count + u4:1;
            let output = regs[4][48:64] ++ regs[5][0:8] ++ CID ++ SID[56:64];
            STATE_RES{state: u4:8, first, second, third, regs, count: new_count, frame, output: output, delay: u4:0, order }
        }
        else if (count == u4:1) {
            let new_count = count + u4:1;
            let output = SID[0:56] ++ order[0:8];
            STATE_RES{state: u4:8, first, second, third, regs, count: new_count, frame, output: output, delay: u4:0, order }
        }
        else if(count == u4:2) {
            let output = order[8:64] ++ regs[5][16:24];
            let new_count = count + u4:1;
            STATE_RES{state: u4:8, first, second, third, regs, count: new_count, frame, output: output, delay: u4:0, order }
        } else if (count == u4:3) {
            let output = regs[5][24:64] ++ regs[6][0:24];
            let new_count = count + u4:1;
            STATE_RES{state: u4:8, first, second, third, regs, count: new_count, frame, output: output, delay: u4:0, order}
        } else {
            let output = regs[6][24:56] ++ FLAGS ++ u24:0x000000;
            let new_order = order + u64:1;
            STATE_RES{state: u4:15, first, second, third, regs, count: u4:0, frame, output: output, delay: u4:0, order: new_order}
        }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }
  else {
      if (delay == u4:7){
        STATE_RES{state: u4:0, first: u64:0, second: u64:0, third: u64:0, regs: [u64:0, u64:0, u64:0, u64:0, u64:0, u64:0, u64:0], count: u4:0, frame, output: u64:0, delay: u4:0, order}
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count: u4:0, frame, output, delay: new_delay, order }
      }
  }
}

proc fsm_proc{
    reading: chan<u8> in;
    output: chan<u8> out;

    init {
        STATE_RES{
          state: u4:0,
          first: u64:0,
          second: u64:0,
          third: u64:0,
          regs: [u64:0, u64:0, u64:0, u64:0, u64:0, u64:0, u64:0],
          count: u4:0,
          frame: false,
          output: u64:0,
          delay: u4:0,
          order: u64:0
        }
    }

    config(reading: chan<u8> in, output: chan<u8> out) {
        (reading, output)
    }

    next(status:STATE_RES) {
        let tok = token();
        let (tok_reading, reading_in) = recv(tok, reading);
        let tok_in = join(tok_reading);

        let new_first = status.first[0:56] ++ reading_in;
        let new_second = status.second[0:56] ++ status.first[56:64];
        let new_third = status.third[0:56] ++ status.second[56:64];
        let output_out = status.output[56:64];
        let new_output = status.output[0:56] ++ u8:0;

        let status = fsm_logic(
          status.state,
          new_first,
          new_second,
          new_third,
          status.regs,
          status.count,
          status.frame,
          new_output,
          status.delay,
          status.order
        );

        send(tok, output, output_out);

        status
    }
}

#[test_proc]
proc proc_fsm_test{

    // Step 1 - Create the channels in reverse order
    reading: chan<u8> out;
    output: chan<u8> in;

    // Step 2 - Add a terminator bool
    terminator: chan<bool> out;

    // Step 3 - Initialize an empty memory for the test proc
    init { () }

    // Step 4 - Config itself only refers to the terminator
    config(terminator: chan<bool> out){

      // Define a channel pair (sender, receiver) to communicate new_work_receiver values as output from the test module
      let(reading, reading_receiver) = chan<u8>("reading");

      // Define a channel pair (sender, receiver) to communicate work_test as an input to the test module
      let(out_test, output) = chan<u8>("output");

      // Spawn a proc that you will test
      spawn fsm_proc(reading_receiver, out_test);

      // Configure the outputs as before
      (reading, output, terminator)

    }

    // Develop test Step 1
    next( state: ()) {

        // Step 2 - Initialize Token
        let tok = token();

        // with tests, you are not testing the actual states themselves
        // you are testing for every control/output transition being valid

        // at tok, receive on output test channel
        // put the data result on output sender



        // Test 2 - send pream
        let tok_b = send(tok, reading, u8:0x55);
        let tok = join(tok_b);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                // Test 2 - send pream
        let tok_b = send(tok, reading, u8:0x55);
        let tok = join(tok_b);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                // Test 2 - send pream
        let tok_b = send(tok, reading, u8:0x55);
        let tok = join(tok_b);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                // Test 2 - send pream
        let tok_b = send(tok, reading, u8:0x55);
        let tok = join(tok_b);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                // Test 2 - send pream
        let tok_b = send(tok, reading, u8:0x55);
        let tok = join(tok_b);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                // Test 2 - send pream
        let tok_b = send(tok, reading, u8:0x55);
        let tok = join(tok_b);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                // Test 2 - send pream
        let tok_b = send(tok, reading, u8:0x55);
        let tok = join(tok_b);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                // Test 2 - send pream
        let tok_b = send(tok, reading, u8:0x5d);
        let tok = join(tok_b);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);

        //Test 3 - send MAC DEST and SRC
        let tok_c = send(tok, reading, u8:0x88);
        let tok = join(tok_c);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                        //Test 3 - send MAC DEST and SRC
        let tok_c = send(tok, reading, u8:0x88);
        let tok = join(tok_c);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 3 - send MAC DEST and SRC
        let tok_c = send(tok, reading, u8:0x88);
        let tok = join(tok_c);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 3 - send MAC DEST and SRC
        let tok_c = send(tok, reading, u8:0x88);
        let tok = join(tok_c);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 3 - send MAC DEST and SRC
        let tok_c = send(tok, reading, u8:0x88);
        let tok = join(tok_c);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 3 - send MAC DEST and SRC
        let tok_c = send(tok, reading, u8:0x88);
        let tok = join(tok_c);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 3 - send MAC DEST and SRC
        let tok_c = send(tok, reading, u8:0x88);
        let tok = join(tok_c);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 3 - send MAC DEST and SRC
        let tok_c = send(tok, reading, u8:0x88);
        let tok = join(tok_c);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);

        //Test 4 - finish MAC SRC and send type and ip header
        let tok_d = send(tok, reading, u8:0x99);
        let tok = join(tok_d);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 4 - finish MAC SRC and send type and ip header
        let tok_d = send(tok, reading, u8:0x99);
        let tok = join(tok_d);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 4 - finish MAC SRC and send type and ip header
        let tok_d = send(tok, reading, u8:0x99);
        let tok = join(tok_d);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 4 - finish MAC SRC and send type and ip header
        let tok_d = send(tok, reading, u8:0x99);
        let tok = join(tok_d);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 4 - finish MAC SRC and send type and ip header
        let tok_d = send(tok, reading, u8:0x99);
        let tok = join(tok_d);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 4 - finish MAC SRC and send type and ip header
        let tok_d = send(tok, reading, u8:0x99);
        let tok = join(tok_d);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 4 - finish MAC SRC and send type and ip header
        let tok_d = send(tok, reading, u8:0x99);
        let tok = join(tok_d);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 4 - finish MAC SRC and send type and ip header
        let tok_d = send(tok, reading, u8:0x99);
        let tok = join(tok_d);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);

        //Test 5 - send type and payload
        let tok_e = send(tok, reading, u8:0x11);
        let tok = join(tok_e);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 5 - send type and payload
        let tok_e = send(tok, reading, u8:0x11);
        let tok = join(tok_e);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 5 - send type and payload
        let tok_e = send(tok, reading, u8:0x11);
        let tok = join(tok_e);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 5 - send type and payload
        let tok_e = send(tok, reading, u8:0x11);
        let tok = join(tok_e);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 5 - send type and payload
        let tok_e = send(tok, reading, u8:0x11);
        let tok = join(tok_e);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 5 - send type and payload
        let tok_e = send(tok, reading, u8:0x11);
        let tok = join(tok_e);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 5 - send type and payload
        let tok_e = send(tok, reading, u8:0x11);
        let tok = join(tok_e);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 5 - send type and payload
        let tok_e = send(tok, reading, u8:0x11);
        let tok = join(tok_e);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);

         //Test 6 - send payload
        let tok_f = send(tok, reading, u8:0x22);
        let tok = join(tok_f);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 6 - send payload
        let tok_f = send(tok, reading, u8:0x22);
        let tok = join(tok_f);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 6 - send payload
        let tok_f = send(tok, reading, u8:0x22);
        let tok = join(tok_f);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 6 - send payload
        let tok_f = send(tok, reading, u8:0x22);
        let tok = join(tok_f);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 6 - send payload
        let tok_f = send(tok, reading, u8:0x22);
        let tok = join(tok_f);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 6 - send payload
        let tok_f = send(tok, reading, u8:0x22);
        let tok = join(tok_f);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 6 - send payload
        let tok_f = send(tok, reading, u8:0x22);
        let tok = join(tok_f);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 6 - send payload
        let tok_f = send(tok, reading, u8:0x22);
        let tok = join(tok_f);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);


        //Test 7 - send payload
        let tok_g = send(tok, reading, u8:0x33);
        let tok = join(tok_g);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 7 - send payload
        let tok_g = send(tok, reading, u8:0x33);
        let tok = join(tok_g);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 7 - send payload
        let tok_g = send(tok, reading, u8:0x33);
        let tok = join(tok_g);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 7 - send payload
        let tok_g = send(tok, reading, u8:0x33);
        let tok = join(tok_g);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 7 - send payload
        let tok_g = send(tok, reading, u8:0x33);
        let tok = join(tok_g);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 7 - send payload
        let tok_g = send(tok, reading, u8:0x33);
        let tok = join(tok_g);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 7 - send payload
        let tok_g = send(tok, reading, u8:0x33);
        let tok = join(tok_g);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 7 - send payload
        let tok_g = send(tok, reading, u8:0x33);
        let tok = join(tok_g);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);


        //Test 8 - send payload
        let tok_h = send(tok, reading, u8:0x44);
        let tok = join(tok_h);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 8 - send payload
        let tok_h = send(tok, reading, u8:0x04);
        let tok = join(tok_h);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 8 - send payload
        let tok_h = send(tok, reading, u8:0x44);
        let tok = join(tok_h);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 8 - send payload
        let tok_h = send(tok, reading, u8:0x44);
        let tok = join(tok_h);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 8 - send payload
        let tok_h = send(tok, reading, u8:0x44);
        let tok = join(tok_h);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 8 - send payload
        let tok_h = send(tok, reading, u8:0x44);
        let tok = join(tok_h);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 8 - send payload
        let tok_h = send(tok, reading, u8:0x04);
        let tok = join(tok_h);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 8 - send payload
        let tok_h = send(tok, reading, u8:0x44);
        let tok = join(tok_h);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);



        //Test 9 - send payload
        let tok_i = send(tok, reading, u8:0x55);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 9 - send payload
        let tok_i = send(tok, reading, u8:0x55);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 9 - send payload
        let tok_i = send(tok, reading, u8:0x55);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 9 - send payload
        let tok_i = send(tok, reading, u8:0x55);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 9 - send payload
        let tok_i = send(tok, reading, u8:0x55);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 9 - send payload
        let tok_i = send(tok, reading, u8:0x55);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 9 - send payload
        let tok_i = send(tok, reading, u8:0x55);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 9 - send payload
        let tok_i = send(tok, reading, u8:0x55);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);



        //Test 10 - send payload
        let tok_i = send(tok, reading, u8:0x56);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 10 - send payload
        let tok_i = send(tok, reading, u8:0x56);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 10 - send payload
        let tok_i = send(tok, reading, u8:0x56);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 10 - send payload
        let tok_i = send(tok, reading, u8:0x56);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 10 - send payload
        let tok_i = send(tok, reading, u8:0x56);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 10 - send payload
        let tok_i = send(tok, reading, u8:0x56);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 10 - send payload
        let tok_i = send(tok, reading, u8:0x56);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 10 - send payload
        let tok_i = send(tok, reading, u8:0x56);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);


                // Test 11 - get results
        let tok_k1 = send(tok, reading, u8:0x66);
        let tok = join(tok_k1);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x55);
                        // Test 11 - get results
        let tok_k1 = send(tok, reading, u8:0x66);
        let tok = join(tok_k1);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x55);
                        // Test 11 - get results
        let tok_k1 = send(tok, reading, u8:0x66);
        let tok = join(tok_k1);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x55);
                        // Test 11 - get results
        let tok_k1 = send(tok, reading, u8:0x66);
        let tok = join(tok_k1);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x55);
                        // Test 11 - get results
        let tok_k1 = send(tok, reading, u8:0x66);
        let tok = join(tok_k1);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x55);
                        // Test 11 - get results
        let tok_k1 = send(tok, reading, u8:0x66);
        let tok = join(tok_k1);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x55);
                        // Test 11 - get results
        let tok_k1 = send(tok, reading, u8:0x66);
        let tok = join(tok_k1);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x55);
                        // Test 11 - get results
        let tok_k1 = send(tok, reading, u8:0x66);
        let tok = join(tok_k1);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x5d);



        // test that the output is 0
                // Test 1 - send 0's
        let tok_k2 = send(tok, reading, u8:13);
        let tok = join(tok_k2);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x88);
        // test that the output is 0
                // Test 1 - send 0's
        let tok_k2 = send(tok, reading, u8:13);
        let tok = join(tok_k2);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x88);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k2 = send(tok, reading, u8:13);
        let tok = join(tok_k2);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x99);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k2 = send(tok, reading, u8:13);
        let tok = join(tok_k2);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x99);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k2 = send(tok, reading, u8:13);
        let tok = join(tok_k2);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x99);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k2 = send(tok, reading, u8:13);
        let tok = join(tok_k2);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x99);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k2 = send(tok, reading, u8:13);
        let tok = join(tok_k2);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x88);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k2 = send(tok, reading, u8:13);
        let tok = join(tok_k2);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x88);
        // test that the output is 0

                // Test 1 - send 0's
        let tok_k3 = send(tok, reading, u8:14);
        let tok = join(tok_k3);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x88);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k3 = send(tok, reading, u8:14);
        let tok = join(tok_k3);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x88);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k3 = send(tok, reading, u8:14);
        let tok = join(tok_k3);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x88);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k3 = send(tok, reading, u8:14);
        let tok = join(tok_k3);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x88);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k3 = send(tok, reading, u8:14);
        let tok = join(tok_k3);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x08);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k3 = send(tok, reading, u8:14);
        let tok = join(tok_k3);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k3 = send(tok, reading, u8:14);
        let tok = join(tok_k3);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x99);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k3 = send(tok, reading, u8:14);
        let tok = join(tok_k3);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x99);
        // test that the output is 0



                // Test 1 - send 0's
        let tok_k4 = send(tok, reading, u8:0);
        let tok = join(tok_k4);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x99);
        // test that the output is 0        // Test 1 - send 0's
                        // Test 1 - send 0's
        let tok_k4 = send(tok, reading, u8:0);
        let tok = join(tok_k4);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x11);
        // test that the output is 0        // Test 1 - send 0's
                        // Test 1 - send 0's
        let tok_k4 = send(tok, reading, u8:0);
        let tok = join(tok_k4);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x11);
        // test that the output is 0        // Test 1 - send 0's
                        // Test 1 - send 0's
        let tok_k4 = send(tok, reading, u8:0);
        let tok = join(tok_k4);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x11);
        // test that the output is 0        // Test 1 - send 0's
                        // Test 1 - send 0's
        let tok_k4 = send(tok, reading, u8:0);
        let tok = join(tok_k4);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x11);
        // test that the output is 0        // Test 1 - send 0's
                        // Test 1 - send 0's
        let tok_k4 = send(tok, reading, u8:0);
        let tok = join(tok_k4);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x11);
        // test that the output is 0        // Test 1 - send 0's
                        // Test 1 - send 0's
        let tok_k4 = send(tok, reading, u8:0);
        let tok = join(tok_k4);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x11);
        // test that the output is 0        // Test 1 - send 0's
                        // Test 1 - send 0's
        let tok_k4 = send(tok, reading, u8:0);
        let tok = join(tok_k4);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x11);
        // test that the output is 0        // Test 1 - send 0's


        let tok_k5 = send(tok, reading, u8:0);
        let tok = join(tok_k5);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x11);
        // test that the output is 0
                let tok_k5 = send(tok, reading, u8:0);
        let tok = join(tok_k5);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x11);
        // test that the output is 0
                let tok_k5 = send(tok, reading, u8:0);
        let tok = join(tok_k5);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x22);
        // test that the output is 0
                let tok_k5 = send(tok, reading, u8:0);
        let tok = join(tok_k5);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x33);
        // test that the output is 0
                let tok_k5 = send(tok, reading, u8:0);
        let tok = join(tok_k5);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x33);
        // test that the output is 0
                let tok_k5 = send(tok, reading, u8:0);
        let tok = join(tok_k5);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x01);
        // test that the output is 0
                let tok_k5 = send(tok, reading, u8:0);
        let tok = join(tok_k5);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x01);
        // test that the output is 0
                let tok_k5 = send(tok, reading, u8:0);
        let tok = join(tok_k5);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x33);
        // test that the output is 0


                // Test 1 - send 0's
        let tok_k6 = send(tok, reading, u8:0);
        let tok = join(tok_k6);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x33);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k6 = send(tok, reading, u8:0);
        let tok = join(tok_k6);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x33);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k6 = send(tok, reading, u8:0);
        let tok = join(tok_k6);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x44);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k6 = send(tok, reading, u8:0);
        let tok = join(tok_k6);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k6 = send(tok, reading, u8:0);
        let tok = join(tok_k6);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k6 = send(tok, reading, u8:0);
        let tok = join(tok_k6);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k6 = send(tok, reading, u8:0);
        let tok = join(tok_k6);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x01);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k6 = send(tok, reading, u8:0);
        let tok = join(tok_k6);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0


                // Test 1 - send 0's
        let tok_k7 = send(tok, reading, u8:0);
        let tok = join(tok_k7);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k7 = send(tok, reading, u8:0);
        let tok = join(tok_k7);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k7 = send(tok, reading, u8:0);
        let tok = join(tok_k7);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k7 = send(tok, reading, u8:0);
        let tok = join(tok_k7);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k7 = send(tok, reading, u8:0);
        let tok = join(tok_k7);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k7 = send(tok, reading, u8:0);
        let tok = join(tok_k7);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k7 = send(tok, reading, u8:0);
        let tok = join(tok_k7);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x02);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k7 = send(tok, reading, u8:0);
        let tok = join(tok_k7);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0


                // Test 1 - send 0's
        let tok_k8 = send(tok, reading, u8:0);
        let tok = join(tok_k8);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k8 = send(tok, reading, u8:0);
        let tok = join(tok_k8);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k8 = send(tok, reading, u8:0);
        let tok = join(tok_k8);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k8 = send(tok, reading, u8:0);
        let tok = join(tok_k8);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k8 = send(tok, reading, u8:0);
        let tok = join(tok_k8);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k8 = send(tok, reading, u8:0);
        let tok = join(tok_k8);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k8 = send(tok, reading, u8:0);
        let tok = join(tok_k8);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k8 = send(tok, reading, u8:0);
        let tok = join(tok_k8);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x44);
        // test that the output is 0


                // Test 1 - send 0's
        let tok_k9 = send(tok, reading, u8:0);
        let tok = join(tok_k9);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x44);
        // test that the output is
                // Test 1 - send 0's
        let tok_k9 = send(tok, reading, u8:0);
        let tok = join(tok_k9);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x04);
        // test that the output is
                // Test 1 - send 0's
        let tok_k9 = send(tok, reading, u8:0);
        let tok = join(tok_k9);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x44);
        // test that the output is
                        // Test 1 - send 0's
        let tok_k9 = send(tok, reading, u8:0);
        let tok = join(tok_k9);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x44);
        // test that the output is
                        // Test 1 - send 0's
        let tok_k9 = send(tok, reading, u8:0);
        let tok = join(tok_k9);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x44);
        // test that the output is
                        // Test 1 - send 0's
        let tok_k9 = send(tok, reading, u8:0);
        let tok = join(tok_k9);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x55);
        // test that the output is
                        // Test 1 - send 0's
        let tok_k9 = send(tok, reading, u8:0);
        let tok = join(tok_k9);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x55);
        // test that the output is
                        // Test 1 - send 0's
        let tok_k9 = send(tok, reading, u8:0);
        let tok = join(tok_k9);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x55);
        // test that the output is


                // Test 1 - send 0's
        let tok_k10 = send(tok, reading, u8:0);
        let tok = join(tok_k10);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x55);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k10 = send(tok, reading, u8:0);
        let tok = join(tok_k10);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x55);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k10 = send(tok, reading, u8:0);
        let tok = join(tok_k10);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x55);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k10 = send(tok, reading, u8:0);
        let tok = join(tok_k10);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x55);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k10 = send(tok, reading, u8:0);
        let tok = join(tok_k10);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k10 = send(tok, reading, u8:0);
        let tok = join(tok_k10);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k10 = send(tok, reading, u8:0);
        let tok = join(tok_k10);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k10 = send(tok, reading, u8:0);
        let tok = join(tok_k10);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0








        // Test 2 - send pream
        let tok_b = send(tok, reading, u8:0x55);
        let tok = join(tok_b);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                // Test 2 - send pream
        let tok_b = send(tok, reading, u8:0x55);
        let tok = join(tok_b);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                // Test 2 - send pream
        let tok_b = send(tok, reading, u8:0x55);
        let tok = join(tok_b);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                // Test 2 - send pream
        let tok_b = send(tok, reading, u8:0x55);
        let tok = join(tok_b);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                // Test 2 - send pream
        let tok_b = send(tok, reading, u8:0x55);
        let tok = join(tok_b);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                // Test 2 - send pream
        let tok_b = send(tok, reading, u8:0x55);
        let tok = join(tok_b);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                // Test 2 - send pream
        let tok_b = send(tok, reading, u8:0x55);
        let tok = join(tok_b);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                // Test 2 - send pream
        let tok_b = send(tok, reading, u8:0x5d);
        let tok = join(tok_b);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);

        //Test 3 - send MAC DEST and SRC
        let tok_c = send(tok, reading, u8:0x88);
        let tok = join(tok_c);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                        //Test 3 - send MAC DEST and SRC
        let tok_c = send(tok, reading, u8:0x88);
        let tok = join(tok_c);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 3 - send MAC DEST and SRC
        let tok_c = send(tok, reading, u8:0x88);
        let tok = join(tok_c);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 3 - send MAC DEST and SRC
        let tok_c = send(tok, reading, u8:0x88);
        let tok = join(tok_c);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 3 - send MAC DEST and SRC
        let tok_c = send(tok, reading, u8:0x88);
        let tok = join(tok_c);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 3 - send MAC DEST and SRC
        let tok_c = send(tok, reading, u8:0x88);
        let tok = join(tok_c);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 3 - send MAC DEST and SRC
        let tok_c = send(tok, reading, u8:0x88);
        let tok = join(tok_c);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 3 - send MAC DEST and SRC
        let tok_c = send(tok, reading, u8:0x88);
        let tok = join(tok_c);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);

        //Test 4 - finish MAC SRC and send type and ip header
        let tok_d = send(tok, reading, u8:0x99);
        let tok = join(tok_d);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 4 - finish MAC SRC and send type and ip header
        let tok_d = send(tok, reading, u8:0x99);
        let tok = join(tok_d);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 4 - finish MAC SRC and send type and ip header
        let tok_d = send(tok, reading, u8:0x99);
        let tok = join(tok_d);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 4 - finish MAC SRC and send type and ip header
        let tok_d = send(tok, reading, u8:0x99);
        let tok = join(tok_d);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 4 - finish MAC SRC and send type and ip header
        let tok_d = send(tok, reading, u8:0x99);
        let tok = join(tok_d);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 4 - finish MAC SRC and send type and ip header
        let tok_d = send(tok, reading, u8:0x99);
        let tok = join(tok_d);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 4 - finish MAC SRC and send type and ip header
        let tok_d = send(tok, reading, u8:0x99);
        let tok = join(tok_d);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 4 - finish MAC SRC and send type and ip header
        let tok_d = send(tok, reading, u8:0x99);
        let tok = join(tok_d);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);

        //Test 5 - send type and payload
        let tok_e = send(tok, reading, u8:0x11);
        let tok = join(tok_e);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 5 - send type and payload
        let tok_e = send(tok, reading, u8:0x11);
        let tok = join(tok_e);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 5 - send type and payload
        let tok_e = send(tok, reading, u8:0x11);
        let tok = join(tok_e);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 5 - send type and payload
        let tok_e = send(tok, reading, u8:0x11);
        let tok = join(tok_e);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 5 - send type and payload
        let tok_e = send(tok, reading, u8:0x11);
        let tok = join(tok_e);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 5 - send type and payload
        let tok_e = send(tok, reading, u8:0x11);
        let tok = join(tok_e);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 5 - send type and payload
        let tok_e = send(tok, reading, u8:0x11);
        let tok = join(tok_e);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 5 - send type and payload
        let tok_e = send(tok, reading, u8:0x11);
        let tok = join(tok_e);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);

         //Test 6 - send payload
        let tok_f = send(tok, reading, u8:0x22);
        let tok = join(tok_f);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 6 - send payload
        let tok_f = send(tok, reading, u8:0x22);
        let tok = join(tok_f);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 6 - send payload
        let tok_f = send(tok, reading, u8:0x22);
        let tok = join(tok_f);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 6 - send payload
        let tok_f = send(tok, reading, u8:0x22);
        let tok = join(tok_f);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 6 - send payload
        let tok_f = send(tok, reading, u8:0x22);
        let tok = join(tok_f);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 6 - send payload
        let tok_f = send(tok, reading, u8:0x22);
        let tok = join(tok_f);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 6 - send payload
        let tok_f = send(tok, reading, u8:0x22);
        let tok = join(tok_f);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 6 - send payload
        let tok_f = send(tok, reading, u8:0x22);
        let tok = join(tok_f);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);


        //Test 7 - send payload
        let tok_g = send(tok, reading, u8:0x33);
        let tok = join(tok_g);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 7 - send payload
        let tok_g = send(tok, reading, u8:0x33);
        let tok = join(tok_g);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 7 - send payload
        let tok_g = send(tok, reading, u8:0x33);
        let tok = join(tok_g);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 7 - send payload
        let tok_g = send(tok, reading, u8:0x33);
        let tok = join(tok_g);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 7 - send payload
        let tok_g = send(tok, reading, u8:0x33);
        let tok = join(tok_g);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 7 - send payload
        let tok_g = send(tok, reading, u8:0x33);
        let tok = join(tok_g);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 7 - send payload
        let tok_g = send(tok, reading, u8:0x33);
        let tok = join(tok_g);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 7 - send payload
        let tok_g = send(tok, reading, u8:0x33);
        let tok = join(tok_g);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);


        //Test 8 - send payload
        let tok_h = send(tok, reading, u8:0x44);
        let tok = join(tok_h);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 8 - send payload
        let tok_h = send(tok, reading, u8:0x04);
        let tok = join(tok_h);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 8 - send payload
        let tok_h = send(tok, reading, u8:0x44);
        let tok = join(tok_h);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 8 - send payload
        let tok_h = send(tok, reading, u8:0x44);
        let tok = join(tok_h);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 8 - send payload
        let tok_h = send(tok, reading, u8:0x44);
        let tok = join(tok_h);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 8 - send payload
        let tok_h = send(tok, reading, u8:0x44);
        let tok = join(tok_h);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 8 - send payload
        let tok_h = send(tok, reading, u8:0x04);
        let tok = join(tok_h);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 8 - send payload
        let tok_h = send(tok, reading, u8:0x44);
        let tok = join(tok_h);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);



        //Test 9 - send payload
        let tok_i = send(tok, reading, u8:0x55);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 9 - send payload
        let tok_i = send(tok, reading, u8:0x55);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 9 - send payload
        let tok_i = send(tok, reading, u8:0x55);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 9 - send payload
        let tok_i = send(tok, reading, u8:0x55);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 9 - send payload
        let tok_i = send(tok, reading, u8:0x55);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 9 - send payload
        let tok_i = send(tok, reading, u8:0x55);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 9 - send payload
        let tok_i = send(tok, reading, u8:0x55);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 9 - send payload
        let tok_i = send(tok, reading, u8:0x55);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);



        //Test 10 - send payload
        let tok_i = send(tok, reading, u8:0x56);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 10 - send payload
        let tok_i = send(tok, reading, u8:0x56);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 10 - send payload
        let tok_i = send(tok, reading, u8:0x56);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 10 - send payload
        let tok_i = send(tok, reading, u8:0x56);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 10 - send payload
        let tok_i = send(tok, reading, u8:0x56);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 10 - send payload
        let tok_i = send(tok, reading, u8:0x56);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 10 - send payload
        let tok_i = send(tok, reading, u8:0x56);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);
                //Test 10 - send payload
        let tok_i = send(tok, reading, u8:0x56);
        let tok = join(tok_i);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x0);


                // Test 11 - get results
        let tok_k1 = send(tok, reading, u8:0x66);
        let tok = join(tok_k1);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x55);
                        // Test 11 - get results
        let tok_k1 = send(tok, reading, u8:0x66);
        let tok = join(tok_k1);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x55);
                        // Test 11 - get results
        let tok_k1 = send(tok, reading, u8:0x66);
        let tok = join(tok_k1);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x55);
                        // Test 11 - get results
        let tok_k1 = send(tok, reading, u8:0x66);
        let tok = join(tok_k1);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x55);
                        // Test 11 - get results
        let tok_k1 = send(tok, reading, u8:0x66);
        let tok = join(tok_k1);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x55);
                        // Test 11 - get results
        let tok_k1 = send(tok, reading, u8:0x66);
        let tok = join(tok_k1);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x55);
                        // Test 11 - get results
        let tok_k1 = send(tok, reading, u8:0x66);
        let tok = join(tok_k1);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x55);
                        // Test 11 - get results
        let tok_k1 = send(tok, reading, u8:0x66);
        let tok = join(tok_k1);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x5d);



        // test that the output is 0
                // Test 1 - send 0's
        let tok_k2 = send(tok, reading, u8:13);
        let tok = join(tok_k2);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x88);
        // test that the output is 0
                // Test 1 - send 0's
        let tok_k2 = send(tok, reading, u8:13);
        let tok = join(tok_k2);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x88);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k2 = send(tok, reading, u8:13);
        let tok = join(tok_k2);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x99);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k2 = send(tok, reading, u8:13);
        let tok = join(tok_k2);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x99);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k2 = send(tok, reading, u8:13);
        let tok = join(tok_k2);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x99);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k2 = send(tok, reading, u8:13);
        let tok = join(tok_k2);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x99);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k2 = send(tok, reading, u8:13);
        let tok = join(tok_k2);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x88);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k2 = send(tok, reading, u8:13);
        let tok = join(tok_k2);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x88);
        // test that the output is 0

                // Test 1 - send 0's
        let tok_k3 = send(tok, reading, u8:14);
        let tok = join(tok_k3);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x88);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k3 = send(tok, reading, u8:14);
        let tok = join(tok_k3);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x88);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k3 = send(tok, reading, u8:14);
        let tok = join(tok_k3);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x88);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k3 = send(tok, reading, u8:14);
        let tok = join(tok_k3);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x88);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k3 = send(tok, reading, u8:14);
        let tok = join(tok_k3);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x08);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k3 = send(tok, reading, u8:14);
        let tok = join(tok_k3);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k3 = send(tok, reading, u8:14);
        let tok = join(tok_k3);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x99);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k3 = send(tok, reading, u8:14);
        let tok = join(tok_k3);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x99);
        // test that the output is 0



                // Test 1 - send 0's
        let tok_k4 = send(tok, reading, u8:0);
        let tok = join(tok_k4);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x99);
        // test that the output is 0        // Test 1 - send 0's
                        // Test 1 - send 0's
        let tok_k4 = send(tok, reading, u8:0);
        let tok = join(tok_k4);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x11);
        // test that the output is 0        // Test 1 - send 0's
                        // Test 1 - send 0's
        let tok_k4 = send(tok, reading, u8:0);
        let tok = join(tok_k4);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x11);
        // test that the output is 0        // Test 1 - send 0's
                        // Test 1 - send 0's
        let tok_k4 = send(tok, reading, u8:0);
        let tok = join(tok_k4);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x11);
        // test that the output is 0        // Test 1 - send 0's
                        // Test 1 - send 0's
        let tok_k4 = send(tok, reading, u8:0);
        let tok = join(tok_k4);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x11);
        // test that the output is 0        // Test 1 - send 0's
                        // Test 1 - send 0's
        let tok_k4 = send(tok, reading, u8:0);
        let tok = join(tok_k4);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x11);
        // test that the output is 0        // Test 1 - send 0's
                        // Test 1 - send 0's
        let tok_k4 = send(tok, reading, u8:0);
        let tok = join(tok_k4);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x11);
        // test that the output is 0        // Test 1 - send 0's
                        // Test 1 - send 0's
        let tok_k4 = send(tok, reading, u8:0);
        let tok = join(tok_k4);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x11);
        // test that the output is 0        // Test 1 - send 0's


        let tok_k5 = send(tok, reading, u8:0);
        let tok = join(tok_k5);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x11);
        // test that the output is 0
                let tok_k5 = send(tok, reading, u8:0);
        let tok = join(tok_k5);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x11);
        // test that the output is 0
                let tok_k5 = send(tok, reading, u8:0);
        let tok = join(tok_k5);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x22);
        // test that the output is 0
                let tok_k5 = send(tok, reading, u8:0);
        let tok = join(tok_k5);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x33);
        // test that the output is 0
                let tok_k5 = send(tok, reading, u8:0);
        let tok = join(tok_k5);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x33);
        // test that the output is 0
                let tok_k5 = send(tok, reading, u8:0);
        let tok = join(tok_k5);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x01);
        // test that the output is 0
                let tok_k5 = send(tok, reading, u8:0);
        let tok = join(tok_k5);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x01);
        // test that the output is 0
                let tok_k5 = send(tok, reading, u8:0);
        let tok = join(tok_k5);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x33);
        // test that the output is 0


                // Test 1 - send 0's
        let tok_k6 = send(tok, reading, u8:0);
        let tok = join(tok_k6);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x33);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k6 = send(tok, reading, u8:0);
        let tok = join(tok_k6);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x33);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k6 = send(tok, reading, u8:0);
        let tok = join(tok_k6);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x44);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k6 = send(tok, reading, u8:0);
        let tok = join(tok_k6);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k6 = send(tok, reading, u8:0);
        let tok = join(tok_k6);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k6 = send(tok, reading, u8:0);
        let tok = join(tok_k6);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k6 = send(tok, reading, u8:0);
        let tok = join(tok_k6);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x01);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k6 = send(tok, reading, u8:0);
        let tok = join(tok_k6);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0


                // Test 1 - send 0's
        let tok_k7 = send(tok, reading, u8:0);
        let tok = join(tok_k7);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k7 = send(tok, reading, u8:0);
        let tok = join(tok_k7);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k7 = send(tok, reading, u8:0);
        let tok = join(tok_k7);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k7 = send(tok, reading, u8:0);
        let tok = join(tok_k7);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k7 = send(tok, reading, u8:0);
        let tok = join(tok_k7);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k7 = send(tok, reading, u8:0);
        let tok = join(tok_k7);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k7 = send(tok, reading, u8:0);
        let tok = join(tok_k7);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x02);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k7 = send(tok, reading, u8:0);
        let tok = join(tok_k7);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x01);
        // test that the output is 0


                // Test 1 - send 0's
        let tok_k8 = send(tok, reading, u8:0);
        let tok = join(tok_k8);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k8 = send(tok, reading, u8:0);
        let tok = join(tok_k8);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k8 = send(tok, reading, u8:0);
        let tok = join(tok_k8);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k8 = send(tok, reading, u8:0);
        let tok = join(tok_k8);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k8 = send(tok, reading, u8:0);
        let tok = join(tok_k8);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k8 = send(tok, reading, u8:0);
        let tok = join(tok_k8);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k8 = send(tok, reading, u8:0);
        let tok = join(tok_k8);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k8 = send(tok, reading, u8:0);
        let tok = join(tok_k8);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x44);
        // test that the output is 0


                // Test 1 - send 0's
        let tok_k9 = send(tok, reading, u8:0);
        let tok = join(tok_k9);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x44);
        // test that the output is
                // Test 1 - send 0's
        let tok_k9 = send(tok, reading, u8:0);
        let tok = join(tok_k9);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x04);
        // test that the output is
                // Test 1 - send 0's
        let tok_k9 = send(tok, reading, u8:0);
        let tok = join(tok_k9);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x44);
        // test that the output is
                        // Test 1 - send 0's
        let tok_k9 = send(tok, reading, u8:0);
        let tok = join(tok_k9);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x44);
        // test that the output is
                        // Test 1 - send 0's
        let tok_k9 = send(tok, reading, u8:0);
        let tok = join(tok_k9);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x44);
        // test that the output is
                        // Test 1 - send 0's
        let tok_k9 = send(tok, reading, u8:0);
        let tok = join(tok_k9);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x55);
        // test that the output is
                        // Test 1 - send 0's
        let tok_k9 = send(tok, reading, u8:0);
        let tok = join(tok_k9);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x55);
        // test that the output is
                        // Test 1 - send 0's
        let tok_k9 = send(tok, reading, u8:0);
        let tok = join(tok_k9);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x55);
        // test that the output is


                // Test 1 - send 0's
        let tok_k10 = send(tok, reading, u8:0);
        let tok = join(tok_k10);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x55);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k10 = send(tok, reading, u8:0);
        let tok = join(tok_k10);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x55);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k10 = send(tok, reading, u8:0);
        let tok = join(tok_k10);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x55);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k10 = send(tok, reading, u8:0);
        let tok = join(tok_k10);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x55);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k10 = send(tok, reading, u8:0);
        let tok = join(tok_k10);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k10 = send(tok, reading, u8:0);
        let tok = join(tok_k10);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k10 = send(tok, reading, u8:0);
        let tok = join(tok_k10);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0
                        // Test 1 - send 0's
        let tok_k10 = send(tok, reading, u8:0);
        let tok = join(tok_k10);
        let (tok, out_sender) = recv(tok, output);
        assert_eq(out_sender, u8:0x00);
        // test that the output is 0



        // Complete the test by send true to the terminator
        let tok = send(tok, terminator, true);

    }

}

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
%%dslx --top=fsm_proc --pipeline_stages=1 --flop_inputs=false --flop_outputs=false --clock_period_ps=4650 --reset=reset
import std;

const E_TYPE = u16:0x0800;
const NEW_ORDER = u8:0x01;
const VERSION = u8:1;
const CID = u32:0x00000001;
const SID = u64:0x0000000000000002;
const FLAGS = u8:0;

struct STATE_RES{
    state:u4,
    first:u64,
    second:u64,
    third:u64,
    regs: u64[7],
    count: u4,
    frame: bool,
    output: u64,
    delay: u4,
    order: u64,
    valid: u1,
    outv: u1
}

fn fsm_logic(
  state: u4,
  first: u64,
  second: u64,
  third: u64,
  regs: u64[7],
  count: u4,
  frame: bool,
  output: u64,
  delay: u4,
  order: u64,
  valid: u1,
  outv: u1) -> STATE_RES {
  if (state == u4:0) {
      if (delay == u4:7) {
            if (first == u64:0x4444444444444444 && valid == u1:1) {
              STATE_RES{state: u4:0, first, second, third, regs, count: u4:0, frame: false, output: u64:0x11, delay: u4:0, order, valid, outv: u1:1}
            } else {
              STATE_RES{state: u4:0, first, second, third, regs, count: u4:0, frame: false, output: u64:0x22, delay: u4:0, order, valid, outv: u1:0}
            }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state: u4:0, first, second, third, regs, count: u4:0, frame: false, output, delay: new_delay, order, valid, outv}
      }
  } else {
      STATE_RES{state: u4:0, first, second, third, regs, count: u4:0, frame: false, output, delay, order, valid, outv}
  }
}



proc fsm_proc{
    reading: chan<u8> in;
    valid: chan<u1> in;
    output: chan<u8> out;
    out_val: chan<u1> out;

    init {
        STATE_RES{
          state: u4:0,
          first: u64:0,
          second: u64:0,
          third: u64:0,
          regs: [u64:0, u64:0, u64:0, u64:0, u64:0, u64:0, u64:0],
          count: u4:0,
          frame: false,
          output: u64:0,
          delay: u4:0,
          order: u64:0,
          valid: u1:0,
          outv: u1:0
        }
    }

    config(reading: chan<u8> in, valid: chan<u1> in, output: chan<u8> out, out_val: chan<u1> out) {
        (reading, valid, output, out_val)
    }

    next(status:STATE_RES) {
        let tok = token();
        let (tok_reading, reading_in) = recv(tok, reading);
        let (tok_valid, valid_in) = recv(tok, valid);
        let tok_in = join(tok_valid, tok_reading);

        let valid = valid_in;
        let new_first = status.first[0:56] ++ reading_in;
        let new_second = status.second[0:56] ++ status.first[56:64];
        let new_third = status.third[0:56] ++ status.second[56:64];
        let output_out = status.output[56:64];
        let new_output = status.output[0:56] ++ u8:0;
        let out_valid = status.outv;

        let status = fsm_logic(
          status.state,
          new_first,
          new_second,
          new_third,
          status.regs,
          status.count,
          status.frame,
          new_output,
          status.delay,
          status.order,
          valid,
          out_valid
        );

        send(tok, output, output_out);
        send(tok, out_val, out_valid);

        status
    }
}

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
%%dslx --top=fsm_proc --pipeline_stages=1 --flop_inputs=false --flop_outputs=false --clock_period_ps=4650 --reset=reset
import std;

const E_TYPE = u16:0x0800;
const NEW_ORDER = u8:0x01;
const VERSION = u8:1;
const CID = u32:0x00000001;
const SID = u64:0x0000000000000002;
const FLAGS = u8:0;

struct STATE_RES{
    state:u4,
    first:u64,
    second:u64,
    third:u64,
    regs: u64[7],
    count: u4,
    frame: bool,
    output: u64,
    delay: u4,
    order: u64
}

fn fsm_logic(
  state: u4,
  first: u64,
  second: u64,
  third: u64,
  regs: u64[7],
  count: u4,
  frame: bool,
  output: u64,
  delay: u4,
  order: u64) -> STATE_RES {
  if (state == u4:0) {
      if(first == u64:0x4444444444444444){
        STATE_RES{state: u4:0, first, second, third, regs, count: u4:0, frame: false, output: u64:0x01, delay, order}
      } else {
          STATE_RES{state: u4:0, first, second, third, regs, count: u4:0, frame: false, output: u64:0x02, delay, order}
      }
  } else {
      STATE_RES{state: u4:0, first, second, third, regs, count: u4:0, frame: false, output: u64:0x4444444444444444, delay, order}
  }
}



proc fsm_proc{
    reading: chan<u8> in;
    output: chan<u8> out;

    init {
        STATE_RES{
          state: u4:0,
          first: u64:0,
          second: u64:0,
          third: u64:0,
          regs: [u64:0, u64:0, u64:0, u64:0, u64:0, u64:0, u64:0],
          count: u4:0,
          frame: false,
          output: u64:0,
          delay: u4:0,
          order: u64:0
        }
    }

    config(reading: chan<u8> in, output: chan<u8> out) {
        (reading, output)
    }

    next(status:STATE_RES) {
        let tok = token();
        let (tok_reading, reading_in) = recv(tok, reading);
        let tok_in = join(tok_reading);

        let new_first = status.first[0:56] ++ reading_in;
        let new_second = status.second[0:56] ++ status.first[56:64];
        let new_third = status.third[0:56] ++ status.second[56:64];
        let output_out = status.output[56:64];
        let new_output = status.output[0:56] ++ u8:0;

        let status = fsm_logic(
          status.state,
          new_first,
          new_second,
          new_third,
          status.regs,
          status.count,
          status.frame,
          new_output,
          status.delay,
          status.order
        );

        send(tok, output, output_out);

        status
    }
}

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
%%dslx --top=fsm_proc --pipeline_stages=1 --flop_inputs=false --flop_outputs=false --clock_period_ps=4650 --reset=reset
import std;
#REAL
const E_TYPE = u16:0x0800;
const NEW_ORDER = u8:0x01;
const VERSION = u8:1;
const CID = u32:0x00000001;
const SID = u64:0x0000000000000002;
const FLAGS = u8:0;

struct STATE_RES{
    state:u4,
    first:u64,
    second:u64,
    third:u64,
    regs: u64[7],
    count: u4,
    frame: bool,
    output: u64,
    delay: u4,
    order: u64
}

fn fsm_logic(
  state: u4,
  first: u64,
  second: u64,
  third: u64,
  regs: u64[7],
  count: u4,
  frame: bool,
  output: u64,
  delay: u4,
  order: u64) -> STATE_RES {
  if (state == u4:0) {
      if (delay == u4:7) {
        if(first == u64:0x555555555555555d){
            STATE_RES{state: u4:1, first, second, third, regs, count: u4:0, frame: false, output: u64:0x0, delay: u4:0, order}
        } else {
            STATE_RES{state: u4:0, first, second, third, regs, count: u4:0, frame: false, output: u64:0x0, delay: u4:0, order}
        }
      }
      else {
          let new_delay = delay + u4:1;
          STATE_RES{state: u4:0, first, second, third, regs, count: u4:0, frame: false, output, delay: new_delay, order}
      }
  }


  else if (state == u4:1) {
      if(delay == u4:7){
        if (count < u4:1) {
            let new_count = u4:1;
            STATE_RES{state, first, second, third, regs, count: new_count, frame, output: u64:0x0, delay: u4:0, order }
        }
        else{
            STATE_RES{state: u4:2, first, second, third, regs, count: u4: 0, frame, output: u64:0x0, delay: u4:0, order }
        }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:2) {
      if (delay == u4:7){
        let new_regs = update(regs, u4:0, third);
        let new_regs = update(new_regs, u4:1, second);
        let new_regs = update(new_regs, u4:2, first);
        STATE_RES{state: u4:3, first, second, third, regs: new_regs, count: u4: 0, frame, output: u64:0, delay: u4:0, order }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:3) {
      if (delay == u4:7){
        if (count < u4:1) {
            let new_count = count + u4:1;
            STATE_RES{state: u4:3, first, second, third, regs, count: new_count, frame, output: u64:0, delay: u4:0, order }
        }
        else{
            let new_regs = update(regs, u4:3, second);
            STATE_RES{state: u4:4, first, second, third, regs: new_regs, count: u4:0, frame, output:u64:0, delay: u4:0, order }
        }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:4) {
      if (delay == u4:7){
        let new_regs = update(regs, u4:4, second);
        let new_regs = update(new_regs, u4:5, first);
        STATE_RES{state: u4:5, first, second, third, regs: new_regs, count: u4:0, frame, output: u64:0, delay: u4:0, order }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:5) {
      if (delay == u4:7){
        if (count < u4:1) {
            let new_count = count + u4:1;
            let new_regs = update(regs, u4:6, first);
            STATE_RES{state: u4:5, first, second, third, regs: new_regs, count: new_count, frame, output: u64:0, delay: u4:0, order }
        }
        else{
            //let output = u64:0x555555555555555d;
            if (regs[5][8:16] != u8:4){
                STATE_RES{state: u4:15, first, second, third, regs, count: u4:0, frame, output: u64:0x888888888888888, delay: u4:0, order }
            }
            else{
                STATE_RES{state: u4:6, first, second, third, regs, count: u4:0, frame, output:u64:0, delay: u4:0, order }
            }
       }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:6) {
      if (delay == u4:7){
        if (count == u4:0) {
            let new_count = count + u4:1;
            //let output = regs[0][48:64] ++ regs[1][0:32] ++ regs[0][0:16];
            STATE_RES{state: u4:6, first, second, third, regs, count: new_count, frame, output: u64:0, delay: u4:0, order }
        }
        else if (count == u4:1) {
            let new_count = count + u4:1;
            //let output = regs[0][16:48] ++ E_TYPE ++ regs[1][32:48];
            STATE_RES{state: u4:6, first, second, third, regs, count: new_count, frame, output: u64:0, delay: u4:0, order }
        }
        else  {
            //let output = regs[1][56:64] ++ regs[2][0:56];
            STATE_RES{state: u4:7, first, second, third, regs, count: u4:0, frame: bool: true, output: u64:0, delay: u4:0, order }
        }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:7) {
      if (delay == u4:7){
        if (frame){
            //let output = regs[2][48:64] ++ regs[3][0:8] ++ regs[4][24:40] ++ NEW_ORDER ++ VERSION ++ regs[4][40:48];
            STATE_RES{state: u4:8, first, second, third, regs, count: u4:0, frame, output: u64:0, delay: u4:0, order }
        } else {
            STATE_RES{state: u4:15, first, second, third, regs, count: u4:0, frame, output: u64:0 , delay: u4:0, order }
        }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:8) {
      if (delay == u4:7){
        if (count == u4:0) {
            let new_count = count + u4:1;
            let output = regs[4][40:64] ++ regs[5][0:8] ++u32:0x00000000;
            STATE_RES{state: u4:8, first, second, third, regs, count: new_count, frame, output: output, delay: u4:0, order }
        }
        else if (count == u4:1) {
            let new_count = count + u4:1;
           //let output = SID[0:56] ++ order[0:8];
            STATE_RES{state: u4:8, first, second, third, regs, count: new_count, frame, output: u64:0, delay: u4:0, order }
        }
        else if(count == u4:2) {
           // let output = order[8:64] ++ regs[5][16:24];
            let new_count = count + u4:1;
            STATE_RES{state: u4:8, first, second, third, regs, count: new_count, frame, output: u64:0, delay: u4:0, order }
        } else if (count == u4:3) {
           // let output = regs[5][24:64] ++ regs[6][0:24];
            let new_count = count + u4:1;
            STATE_RES{state: u4:8, first, second, third, regs, count: new_count, frame, output: u64:0, delay: u4:0, order}
        } else {
           // let output = regs[6][24:56] ++ FLAGS ++ u24:0x000000;
            let new_order = order + u64:1;
            STATE_RES{state: u4:15, first, second, third, regs, count: u4:0, frame, output: u64:0, delay: u4:0, order: new_order}
        }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }
  else {
      if (delay == u4:7){
        STATE_RES{state: u4:0, first: u64:0, second: u64:0, third: u64:0, regs: [u64:0, u64:0, u64:0, u64:0, u64:0, u64:0, u64:0], count: u4:0, frame, output: u64:0, delay: u4:0, order}
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count: u4:0, frame, output, delay: new_delay, order }
      }
  }
}

proc fsm_proc{
    reading: chan<u8> in;
    output: chan<u8> out;

    init {
        STATE_RES{
          state: u4:0,
          first: u64:0,
          second: u64:0,
          third: u64:0,
          regs: [u64:0, u64:0, u64:0, u64:0, u64:0, u64:0, u64:0],
          count: u4:0,
          frame: false,
          output: u64:0,
          delay: u4:0,
          order: u64:0
        }
    }

    config(reading: chan<u8> in, output: chan<u8> out) {
        (reading, output)
    }

    next(status:STATE_RES) {
        let tok = token();
        let (tok_reading, reading_in) = recv(tok, reading);
        let tok_in = join(tok_reading);

        let new_first = status.first[0:56] ++ reading_in;
        let new_second = status.second[0:56] ++ status.first[56:64];
        let new_third = status.third[0:56] ++ status.second[56:64];
        let output_out = status.output[56:64];
        let new_output = status.output[0:56] ++ u8:0;

        let status = fsm_logic(
          status.state,
          new_first,
          new_second,
          new_third,
          status.regs,
          status.count,
          status.frame,
          new_output,
          status.delay,
          status.order
        );

        send(tok, output, output_out);

        status
    }
}


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
%%dslx --top=fsm_proc --pipeline_stages=1 --flop_inputs=false --flop_outputs=false --clock_period_ps=4650 --reset=reset
import std;
#1s
const E_TYPE = u16:0x0800;
const NEW_ORDER = u8:0x01;
const VERSION = u8:1;
const CID = u32:0x00000001;
const SID = u64:0x0000000000000002;
const FLAGS = u8:0;

struct STATE_RES{
    state:u4,
    first:u64,
    second:u64,
    third:u64,
    regs: u64[7],
    count: u4,
    frame: bool,
    output: u64,
    delay: u4,
    order: u64
}

fn fsm_logic(
  state: u4,
  first: u64,
  second: u64,
  third: u64,
  regs: u64[7],
  count: u4,
  frame: bool,
  output: u64,
  delay: u4,
  order: u64) -> STATE_RES {
  if (state == u4:0) {
      if (delay == u4:7) {
        if(first == u64:0x555555555555555d){
            STATE_RES{state: u4:1, first, second, third, regs, count: u4:0, frame: false, output: u64:0x0, delay: u4:0, order}
        } else {
            STATE_RES{state: u4:0, first, second, third, regs, count: u4:0, frame: false, output: u64:0x1111111111111111, delay: u4:0, order}
        }
      }
      else {
          let new_delay = delay + u4:1;
          STATE_RES{state: u4:0, first, second, third, regs, count: u4:0, frame: false, output, delay: new_delay, order}
      }
  }


  else if (state == u4:1) {
      if(delay == u4:7){
        if (count < u4:1) {
            let new_count = u4:1;
            STATE_RES{state, first, second, third, regs, count: new_count, frame, output: u64:0x0, delay: u4:0, order }
        }
        else{
            STATE_RES{state: u4:2, first, second, third, regs, count: u4: 0, frame, output: u64:0x0, delay: u4:0, order }
        }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:2) {
      if (delay == u4:7){
        let new_regs = update(regs, u4:0, third);
        let new_regs = update(new_regs, u4:1, second);
        let new_regs = update(new_regs, u4:2, first);
        STATE_RES{state: u4:3, first, second, third, regs: new_regs, count: u4: 0, frame, output: u64:0, delay: u4:0, order }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:3) {
      if (delay == u4:7){
        if (count < u4:1) {
            let new_count = count + u4:1;
            STATE_RES{state: u4:3, first, second, third, regs, count: new_count, frame, output: u64:0, delay: u4:0, order }
        }
        else{
            let new_regs = update(regs, u4:3, second);
            STATE_RES{state: u4:4, first, second, third, regs: new_regs, count: u4:0, frame, output:u64:0, delay: u4:0, order }
        }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:4) {
      if (delay == u4:7){
        let new_regs = update(regs, u4:4, second);
        let new_regs = update(new_regs, u4:5, first);
        STATE_RES{state: u4:5, first, second, third, regs: new_regs, count: u4:0, frame, output: u64:0, delay: u4:0, order }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:5) {
      if (delay == u4:7){
        if (count < u4:1) {
            let new_count = count + u4:1;
            let new_regs = update(regs, u4:6, first);
            STATE_RES{state: u4:5, first, second, third, regs: new_regs, count: new_count, frame, output: u64:0, delay: u4:0, order }
        }
        else{
            //let output = u64:0x555555555555555d;
            if (regs[5][8:16] != u8:4){
                STATE_RES{state: u4:15, first, second, third, regs, count: u4:0, frame, output: u64:0x888888888888888, delay: u4:0, order }
            }
            else{
                STATE_RES{state: u4:6, first, second, third, regs, count: u4:0, frame, output:u64:0, delay: u4:0, order }
            }
       }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:6) {
      if (delay == u4:7){
        if (count == u4:0) {
            let new_count = count + u4:1;
            //let output = regs[0][48:64] ++ regs[1][0:32] ++ regs[0][0:16];
            STATE_RES{state: u4:6, first, second, third, regs, count: new_count, frame, output: u64:0, delay: u4:0, order }
        }
        else if (count == u4:1) {
            let new_count = count + u4:1;
            //let output = regs[0][16:48] ++ E_TYPE ++ regs[1][32:48];
            STATE_RES{state: u4:6, first, second, third, regs, count: new_count, frame, output: u64:0, delay: u4:0, order }
        }
        else  {
            //let output = regs[1][56:64] ++ regs[2][0:56];
            STATE_RES{state: u4:7, first, second, third, regs, count: u4:0, frame: bool: true, output: u64:0, delay: u4:0, order }
        }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:7) {
      if (delay == u4:7){
        if (frame){
            //let output = regs[2][48:64] ++ regs[3][0:8] ++ regs[4][24:40] ++ NEW_ORDER ++ VERSION ++ regs[4][40:48];
            STATE_RES{state: u4:8, first, second, third, regs, count: u4:0, frame, output: u64:0, delay: u4:0, order }
        } else {
            STATE_RES{state: u4:15, first, second, third, regs, count: u4:0, frame, output: u64:0 , delay: u4:0, order }
        }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:8) {
      if (delay == u4:7){
        if (count == u4:0) {
            let new_count = count + u4:1;
            let output = regs[4][40:64] ++ regs[5][0:8] ++u32:0x00000000;
            STATE_RES{state: u4:8, first, second, third, regs, count: new_count, frame, output: output, delay: u4:0, order }
        }
        else if (count == u4:1) {
            let new_count = count + u4:1;
           //let output = SID[0:56] ++ order[0:8];
            STATE_RES{state: u4:8, first, second, third, regs, count: new_count, frame, output: u64:0, delay: u4:0, order }
        }
        else if(count == u4:2) {
           // let output = order[8:64] ++ regs[5][16:24];
            let new_count = count + u4:1;
            STATE_RES{state: u4:8, first, second, third, regs, count: new_count, frame, output: u64:0, delay: u4:0, order }
        } else if (count == u4:3) {
           // let output = regs[5][24:64] ++ regs[6][0:24];
            let new_count = count + u4:1;
            STATE_RES{state: u4:8, first, second, third, regs, count: new_count, frame, output: u64:0, delay: u4:0, order}
        } else {
           // let output = regs[6][24:56] ++ FLAGS ++ u24:0x000000;
            let new_order = order + u64:1;
            STATE_RES{state: u4:15, first, second, third, regs, count: u4:0, frame, output: u64:0, delay: u4:0, order: new_order}
        }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }
  else {
      if (delay == u4:7){
        STATE_RES{state: u4:0, first: u64:0, second: u64:0, third: u64:0, regs: [u64:0, u64:0, u64:0, u64:0, u64:0, u64:0, u64:0], count: u4:0, frame, output: u64:0, delay: u4:0, order}
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count: u4:0, frame, output, delay: new_delay, order }
      }
  }
}

proc fsm_proc{
    reading: chan<u8> in;
    output: chan<u8> out;

    init {
        STATE_RES{
          state: u4:0,
          first: u64:0,
          second: u64:0,
          third: u64:0,
          regs: [u64:0, u64:0, u64:0, u64:0, u64:0, u64:0, u64:0],
          count: u4:0,
          frame: false,
          output: u64:0,
          delay: u4:0,
          order: u64:0
        }
    }

    config(reading: chan<u8> in, output: chan<u8> out) {
        (reading, output)
    }

    next(status:STATE_RES) {
        let tok = token();
        let (tok_reading, reading_in) = recv(tok, reading);
        let tok_in = join(tok_reading);

        let new_first = status.first[0:56] ++ reading_in;
        let new_second = status.second[0:56] ++ status.first[56:64];
        let new_third = status.third[0:56] ++ status.second[56:64];
        let output_out = status.output[56:64];
        let new_output = status.output[0:56] ++ u8:0;

        let status = fsm_logic(
          status.state,
          new_first,
          new_second,
          new_third,
          status.regs,
          status.count,
          status.frame,
          new_output,
          status.delay,
          status.order
        );

        send(tok, output, output_out);

        status
    }
}


In [ ]:
%%dslx --top=fsm_proc --pipeline_stages=1 --flop_inputs=false --flop_outputs=false --clock_period_ps=4650 --reset=reset
import std;
const E_TYPE = u16:0x0800;
const NEW_ORDER = u8:0x01;
const VERSION = u8:1;
const CID = u32:0x00000001;
const SID = u64:0x0000000000000002;
const FLAGS = u8:0;

struct STATE_RES{
    state:u4,
    first:u64,
    second:u64,
    third:u64,
    regs: u64[7],
    count: u4,
    frame: bool,
    output: u64,
    delay: u4,
    order: u64
}

fn fsm_logic(
  state: u4,
  first: u64,
  second: u64,
  third: u64,
  regs: u64[7],
  count: u4,
  frame: bool,
  output: u64,
  delay: u4,
  order: u64) -> STATE_RES {
  if (state == u4:0) {
      if (delay == u4:7) {
        if(first == u64:0x555555555555555d){
            STATE_RES{state: u4:1, first, second, third, regs, count: u4:0, frame: false, output: u64:0x0, delay: u4:0, order}
        } else {
            STATE_RES{state: u4:0, first, second, third, regs, count: u4:0, frame: false, output: u64:0x1, delay: u4:0, order}
        }
      }
      else {
          let new_delay = delay + u4:1;
          STATE_RES{state: u4:0, first, second, third, regs, count: u4:0, frame: false, output, delay: new_delay, order}
      }
  }


  else if (state == u4:1) {
      if(delay == u4:7){
        if (count < u4:1) {
            let new_count = u4:1;
            STATE_RES{state, first, second, third, regs, count: new_count, frame, output: u64:0x1, delay: u4:0, order }
        }
        else{
            STATE_RES{state: u4:2, first, second, third, regs, count: u4: 0, frame, output: u64:0x1, delay: u4:0, order }
        }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:2) {
      if (delay == u4:7){
        let new_regs = update(regs, u4:0, third);
        let new_regs = update(new_regs, u4:1, second);
        let new_regs = update(new_regs, u4:2, first);
        STATE_RES{state: u4:3, first, second, third, regs: new_regs, count: u4: 0, frame, output: u64:1, delay: u4:0, order }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:3) {
      if (delay == u4:7){
        if (count < u4:1) {
            let new_count = count + u4:1;
            STATE_RES{state: u4:3, first, second, third, regs, count: new_count, frame, output: u64:1, delay: u4:0, order }
        }
        else{
            let new_regs = update(regs, u4:3, second);
            STATE_RES{state: u4:4, first, second, third, regs: new_regs, count: u4:0, frame, output:u64:1, delay: u4:0, order }
        }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:4) {
      if (delay == u4:7){
        let new_regs = update(regs, u4:4, second);
        let new_regs = update(new_regs, u4:5, first);
        STATE_RES{state: u4:5, first, second, third, regs: new_regs, count: u4:0, frame, output: u64:1, delay: u4:0, order }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:5) {
      if (delay == u4:7){
        if (count < u4:1) {
            let new_count = count + u4:1;
            let new_regs = update(regs, u4:6, first);
            STATE_RES{state: u4:5, first, second, third, regs: new_regs, count: new_count, frame, output: u64:1, delay: u4:0, order }
        }
        else{
            //let output = u64:0x555555555555555d;
            if (regs[5][8:16] != u8:4){
                STATE_RES{state: u4:15, first, second, third, regs, count: u4:0, frame, output: u64:0x888888888888888, delay: u4:0, order }
            }
            else{
                STATE_RES{state: u4:6, first, second, third, regs, count: u4:0, frame, output:u64:1, delay: u4:0, order }
            }
       }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:6) {
      if (delay == u4:7){
        if (count == u4:0) {
            let new_count = count + u4:1;
            //let output = regs[0][48:64] ++ regs[1][0:32] ++ regs[0][0:16];
            STATE_RES{state: u4:6, first, second, third, regs, count: new_count, frame, output: u64:1, delay: u4:0, order }
        }
        else if (count == u4:1) {
            let new_count = count + u4:1;
            //let output = regs[0][16:48] ++ E_TYPE ++ regs[1][32:48];
            STATE_RES{state: u4:6, first, second, third, regs, count: new_count, frame, output: u64:1, delay: u4:0, order }
        }
        else  {
            //let output = regs[1][56:64] ++ regs[2][0:56];
            STATE_RES{state: u4:7, first, second, third, regs, count: u4:0, frame: bool: true, output: u64:1, delay: u4:0, order }
        }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:7) {
      if (delay == u4:7){
        if (frame){
            //let output = regs[2][48:64] ++ regs[3][0:8] ++ regs[4][24:40] ++ NEW_ORDER ++ VERSION ++ regs[4][40:48];
            STATE_RES{state: u4:8, first, second, third, regs, count: u4:0, frame, output: u64:1, delay: u4:0, order }
        } else {
            STATE_RES{state: u4:15, first, second, third, regs, count: u4:0, frame, output: u64:1 , delay: u4:0, order }
        }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }

  else if (state == u4:8) {
      if (delay == u4:7){
        if (count == u4:0) {
            let new_count = count + u4:1;
            let output = regs[4][40:64] ++ regs[5][0:8] ++u32:0x00000000;
            STATE_RES{state: u4:8, first, second, third, regs, count: new_count, frame, output: output, delay: u4:0, order }
        }
        else if (count == u4:1) {
            let new_count = count + u4:1;
           //let output = SID[0:56] ++ order[0:8];
            STATE_RES{state: u4:8, first, second, third, regs, count: new_count, frame, output: u64:0, delay: u4:0, order }
        }
        else if(count == u4:2) {
           // let output = order[8:64] ++ regs[5][16:24];
            let new_count = count + u4:1;
            STATE_RES{state: u4:8, first, second, third, regs, count: new_count, frame, output: u64:0, delay: u4:0, order }
        } else if (count == u4:3) {
           // let output = regs[5][24:64] ++ regs[6][0:24];
            let new_count = count + u4:1;
            STATE_RES{state: u4:8, first, second, third, regs, count: new_count, frame, output: u64:0, delay: u4:0, order}
        } else {
           // let output = regs[6][24:56] ++ FLAGS ++ u24:0x000000;
            let new_order = order + u64:1;
            STATE_RES{state: u4:15, first, second, third, regs, count: u4:0, frame, output: u64:0, delay: u4:0, order: new_order}
        }
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count, frame, output, delay: new_delay, order }
      }
  }
  else {
      if (delay == u4:7){
        STATE_RES{state: u4:0, first: u64:0, second: u64:0, third: u64:0, regs: [u64:0, u64:0, u64:0, u64:0, u64:0, u64:0, u64:0], count: u4:0, frame, output: u64:0, delay: u4:0, order}
      } else {
          let new_delay = delay + u4:1;
          STATE_RES{state, first, second, third, regs, count: u4:0, frame, output, delay: new_delay, order }
      }
  }
}

proc fsm_proc{
    reading: chan<u8> in;
    output: chan<u8> out;

    init {
        STATE_RES{
          state: u4:0,
          first: u64:0,
          second: u64:0,
          third: u64:0,
          regs: [u64:0, u64:0, u64:0, u64:0, u64:0, u64:0, u64:0],
          count: u4:0,
          frame: false,
          output: u64:0,
          delay: u4:0,
          order: u64:0
        }
    }

    config(reading: chan<u8> in, output: chan<u8> out) {
        (reading, output)
    }

    next(status:STATE_RES) {
        let tok = token();
        let (tok_reading, reading_in) = recv(tok, reading);
        let tok_in = join(tok_reading);

        let new_first = status.first[0:56] ++ reading_in;
        let new_second = status.second[0:56] ++ status.first[56:64];
        let new_third = status.third[0:56] ++ status.second[56:64];
        let output_out = status.output[56:64];
        let new_output = status.output[0:56] ++ u8:0;

        let status = fsm_logic(
          status.state,
          new_first,
          new_second,
          new_third,
          status.regs,
          status.count,
          status.frame,
          new_output,
          status.delay,
          status.order
        );

        send(tok, output, output_out);

        status
    }
}


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>